In [ ]:
%

In [5]:
import numpy as np
from itertools import combinations
from scipy.linalg import orth
from scipy.optimize import linprog

def is_linearly_independent(vectors):
    return np.linalg.matrix_rank(vectors) == len(vectors)

def closure(matrix, subset):
    n = matrix.shape[1]
    current_set = set(subset)
    current_rank = np.linalg.matrix_rank(matrix[:, list(current_set)])
    for i in range(n):
        if i in current_set:
            continue
        new_set = current_set | {i}
        new_rank = np.linalg.matrix_rank(matrix[:, list(new_set)])
        if new_rank == current_rank:
            current_set.add(i)
    return current_set

def find_flats(matrix):
    n = matrix.shape[1]
    flats = set()
    for r in range(1,n + 1):
        for subset in combinations(range(n), r):
            cl = closure(matrix, subset)
            flats.add(tuple(sorted(cl)))
    return [set(flat) for flat in flats]

def generate_flags(flats):
    flats_sorted = sorted(flats, key=lambda x: (len(x), sorted(x)))
    flags = []
    def dfs(current_flag, remaining_flats):
        for i, flat in enumerate(remaining_flats):
            if not current_flag or all(f in flat for f in current_flag[-1]) and len(flat) > len(current_flag[-1]):
                new_flag = current_flag + [flat]
                flags.append(new_flag)
                dfs(new_flag, remaining_flats[i+1:])
    dfs([], flats_sorted)
    return flags

def project_to_quotient(flat, total_elements):
    vector = [0] * total_elements
    for element in flat:
        vector[element] = 1
    return vector

def fan_map(bergman_fan, V):
    cones = []
    for flag in bergman_fan:
        cone_generators = [project_to_quotient(F, V.shape[1]) for F in flag]
        mapped_cone = [V.dot(np.array(generator)) for generator in cone_generators]
        cones.append(mapped_cone)
    return cones

def find_extreme_rays(cones):
    extreme_rays = set()
    for cone in cones:
        for vector in cone:
            normalized = tuple(np.round(vector / np.linalg.norm(vector), 8))
            extreme_rays.add(normalized)
    return [np.array(ray) for ray in extreme_rays]

# Example usage
matrix = np.array([
    [-4, 0, 0],
    [0, -4, 0],
    [0, 0, -4],
    [2, 2, 4],
    [2, 4, 2],
    [-2, -2, 4],
    [2, 4, -2]
]).T

V = np.array([
    [4, 0, 0, -2, -2, -2, 2],
    [0, 4, 0, -2, -4, -2, 4],
    [0, 0, 4, -4, -2, 4, -2]
])

flats = find_flats(matrix)
flags = generate_flags(flats)
bergman_fan = flags
mapped_cones = fan_map(bergman_fan, V)
extreme_rays = find_extreme_rays(mapped_cones)

print("Found extreme rays:", len(extreme_rays))
for ray in extreme_rays:
    print(ray)

Found extreme rays: 91
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[ 0.40824829 -0.81649658 -0.40824829]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[-0.40824829  0.40824829  0.81649658]
[0.70710678 0.70710678 0.        ]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[-0.42640143 -0.63960215 -0.63960215]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[-0.40824829  0.40824829 -0.81649658]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[-0.40824829 -0.81649658 -0.40824829]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[ 0.40824829 -0.40824829  0.81649658]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[0. 1. 0.]
[nan nan nan]
[ 0.40824829  0.81649658 -0.40824829]
[nan nan nan]
[0. 0. 1.]
[nan nan nan]
[ 0.40824829 -0.40824829 -0.81649658]
[nan nan nan]
[nan nan nan]


C:\Users\dpomi\AppData\Local\Temp\ipykernel_24284\3150505881.py:61: RuntimeWarning: invalid value encountered in divide
  normalized = tuple(np.round(vector / np.linalg.norm(vector), 8))
